# Laplace Equation in Electrostatics

## Theoretical Background

The **Laplace equation** is one of the most important partial differential equations in mathematical physics. In electrostatics, it describes the electric potential $\phi$ in a charge-free region:

$$\nabla^2 \phi = 0$$

In two dimensions with Cartesian coordinates, this becomes:

$$\frac{\partial^2 \phi}{\partial x^2} + \frac{\partial^2 \phi}{\partial y^2} = 0$$

### Physical Interpretation

The Laplace equation arises from **Gauss's law** for electrostatics:

$$\nabla \cdot \mathbf{E} = \frac{\rho}{\epsilon_0}$$

where $\mathbf{E}$ is the electric field, $\rho$ is the charge density, and $\epsilon_0$ is the permittivity of free space. Since the electric field is related to the potential by $\mathbf{E} = -\nabla \phi$, we obtain:

$$\nabla^2 \phi = -\frac{\rho}{\epsilon_0}$$

This is **Poisson's equation**. In regions where $\rho = 0$ (no free charges), it reduces to the Laplace equation.

### Numerical Solution: Finite Difference Method

We solve the Laplace equation numerically using the **finite difference method** with the **relaxation (Jacobi) algorithm**. Discretizing the domain on a grid with spacing $h$, the Laplacian is approximated as:

$$\nabla^2 \phi \approx \frac{\phi_{i+1,j} + \phi_{i-1,j} + \phi_{i,j+1} + \phi_{i,j-1} - 4\phi_{i,j}}{h^2} = 0$$

Rearranging gives the iterative update rule:

$$\phi_{i,j}^{(n+1)} = \frac{1}{4}\left(\phi_{i+1,j}^{(n)} + \phi_{i-1,j}^{(n)} + \phi_{i,j+1}^{(n)} + \phi_{i,j-1}^{(n)}\right)$$

### Problem Setup

We will solve a classic boundary value problem: a **rectangular conducting box** with specified potentials on each boundary. This simulates a parallel-plate capacitor configuration:

- Top boundary: $\phi = V_0$ (high potential)
- Bottom boundary: $\phi = 0$ (grounded)
- Left and right boundaries: $\phi = 0$ (grounded)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm

# Domain parameters
Lx = 1.0  # Length in x-direction (meters)
Ly = 1.0  # Length in y-direction (meters)
Nx = 50   # Number of grid points in x
Ny = 50   # Number of grid points in y

# Grid spacing
dx = Lx / (Nx - 1)
dy = Ly / (Ny - 1)

# Create coordinate arrays
x = np.linspace(0, Lx, Nx)
y = np.linspace(0, Ly, Ny)
X, Y = np.meshgrid(x, y)

# Initialize potential array
phi = np.zeros((Ny, Nx))

# Boundary conditions
V0 = 100.0  # Potential at top boundary (Volts)

# Apply boundary conditions
phi[0, :] = 0      # Bottom boundary (y = 0)
phi[-1, :] = V0    # Top boundary (y = Ly)
phi[:, 0] = 0      # Left boundary (x = 0)
phi[:, -1] = 0     # Right boundary (x = Lx)

print(f"Grid size: {Nx} x {Ny}")
print(f"Grid spacing: dx = {dx:.4f} m, dy = {dy:.4f} m")
print(f"Boundary conditions: V_top = {V0} V, V_bottom = V_left = V_right = 0 V")

## Jacobi Relaxation Method

We implement the iterative Jacobi method to solve the Laplace equation. The algorithm converges when the maximum change in potential between iterations falls below a specified tolerance $\epsilon$:

$$\max_{i,j} |\phi_{i,j}^{(n+1)} - \phi_{i,j}^{(n)}| < \epsilon$$

In [ ]:
def laplace_jacobi(phi, tolerance=1e-5, max_iterations=10000):
    """
    Solve the 2D Laplace equation using Jacobi relaxation method.
    
    Parameters:
    -----------
    phi : ndarray
        Initial potential array with boundary conditions applied
    tolerance : float
        Convergence criterion
    max_iterations : int
        Maximum number of iterations
    
    Returns:
    --------
    phi : ndarray
        Converged potential distribution
    iterations : int
        Number of iterations performed
    residuals : list
        History of maximum residuals
    """
    phi = phi.copy()
    residuals = []
    
    for iteration in range(max_iterations):
        phi_old = phi.copy()
        
        # Update interior points using Jacobi formula
        phi[1:-1, 1:-1] = 0.25 * (
            phi_old[2:, 1:-1] +    # phi(i+1, j)
            phi_old[:-2, 1:-1] +   # phi(i-1, j)
            phi_old[1:-1, 2:] +    # phi(i, j+1)
            phi_old[1:-1, :-2]     # phi(i, j-1)
        )
        
        # Calculate maximum change (residual)
        residual = np.max(np.abs(phi - phi_old))
        residuals.append(residual)
        
        # Check convergence
        if residual < tolerance:
            print(f"Converged after {iteration + 1} iterations")
            print(f"Final residual: {residual:.2e}")
            return phi, iteration + 1, residuals
    
    print(f"Warning: Did not converge after {max_iterations} iterations")
    print(f"Final residual: {residual:.2e}")
    return phi, max_iterations, residuals

# Solve the Laplace equation
phi_solution, num_iterations, residual_history = laplace_jacobi(phi)

## Convergence Analysis

Let's examine the convergence behavior of the Jacobi method:

In [ ]:
plt.figure(figsize=(10, 4))
plt.semilogy(residual_history)
plt.xlabel('Iteration', fontsize=12)
plt.ylabel('Maximum Residual', fontsize=12)
plt.title('Convergence of Jacobi Relaxation Method', fontsize=14)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Electric Field Calculation

The electric field is computed from the potential using:

$$\mathbf{E} = -\nabla \phi = -\frac{\partial \phi}{\partial x}\hat{\mathbf{x}} - \frac{\partial \phi}{\partial y}\hat{\mathbf{y}}$$

We use central differences for numerical differentiation.

In [ ]:
# Calculate electric field components using central differences
Ey, Ex = np.gradient(-phi_solution, dy, dx)

# Calculate electric field magnitude
E_magnitude = np.sqrt(Ex**2 + Ey**2)

print(f"Maximum electric field magnitude: {np.max(E_magnitude):.2f} V/m")
print(f"Minimum electric field magnitude: {np.min(E_magnitude):.2f} V/m")

## Visualization

We create a comprehensive visualization showing:
1. **Electric potential** as a color map with equipotential contours
2. **Electric field** as vector arrows (streamlines)

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Plot 1: Electric Potential with Equipotential Lines
ax1 = axes[0]
im1 = ax1.pcolormesh(X, Y, phi_solution, cmap='RdYlBu_r', shading='auto')
contours = ax1.contour(X, Y, phi_solution, levels=15, colors='black', linewidths=0.5)
ax1.clabel(contours, inline=True, fontsize=8, fmt='%.0f V')
cbar1 = fig.colorbar(im1, ax=ax1, label='Potential (V)')
ax1.set_xlabel('x (m)', fontsize=12)
ax1.set_ylabel('y (m)', fontsize=12)
ax1.set_title('Electric Potential $\\phi(x, y)$', fontsize=14)
ax1.set_aspect('equal')

# Plot 2: Electric Field with Streamlines
ax2 = axes[1]
im2 = ax2.pcolormesh(X, Y, E_magnitude, cmap='viridis', shading='auto')
cbar2 = fig.colorbar(im2, ax=ax2, label='|E| (V/m)')

# Add streamlines for field direction
# Use a subset of points for better visualization
skip = 3
ax2.streamplot(X, Y, Ex, Ey, color='white', density=1.5, linewidth=0.8, arrowsize=0.8)

ax2.set_xlabel('x (m)', fontsize=12)
ax2.set_ylabel('y (m)', fontsize=12)
ax2.set_title('Electric Field $\\mathbf{E}(x, y)$', fontsize=14)
ax2.set_aspect('equal')

plt.tight_layout()
plt.savefig('plot.png', dpi=150, bbox_inches='tight')
plt.show()

print("\nPlot saved to 'plot.png'")

## Analytical Verification

For this boundary value problem, an analytical solution exists as a Fourier series:

$$\phi(x, y) = \sum_{n=1,3,5,...}^{\infty} \frac{4V_0}{n\pi} \cdot \frac{\sinh(n\pi y/L_x)}{\sinh(n\pi L_y/L_x)} \cdot \sin\left(\frac{n\pi x}{L_x}\right)$$

Let's compare our numerical solution with the analytical result:

In [ ]:
def analytical_solution(X, Y, V0, Lx, Ly, n_terms=50):
    """
    Compute the analytical solution using Fourier series.
    
    Parameters:
    -----------
    X, Y : ndarray
        Meshgrid coordinate arrays
    V0 : float
        Potential at top boundary
    Lx, Ly : float
        Domain dimensions
    n_terms : int
        Number of terms in Fourier series
    
    Returns:
    --------
    phi_analytical : ndarray
        Analytical potential distribution
    """
    phi_analytical = np.zeros_like(X)
    
    for n in range(1, 2*n_terms, 2):  # Odd terms only
        coefficient = 4 * V0 / (n * np.pi)
        sinh_ratio = np.sinh(n * np.pi * Y / Lx) / np.sinh(n * np.pi * Ly / Lx)
        sin_term = np.sin(n * np.pi * X / Lx)
        phi_analytical += coefficient * sinh_ratio * sin_term
    
    return phi_analytical

# Calculate analytical solution
phi_analytical = analytical_solution(X, Y, V0, Lx, Ly)

# Compute error
error = np.abs(phi_solution - phi_analytical)
max_error = np.max(error)
mean_error = np.mean(error)
rms_error = np.sqrt(np.mean(error**2))

print("Comparison with Analytical Solution:")
print(f"Maximum absolute error: {max_error:.4f} V")
print(f"Mean absolute error: {mean_error:.4f} V")
print(f"RMS error: {rms_error:.4f} V")
print(f"Relative error (max): {100*max_error/V0:.3f}%")

In [ ]:
# Visualize the error distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Numerical solution
im1 = axes[0].pcolormesh(X, Y, phi_solution, cmap='RdYlBu_r', shading='auto')
fig.colorbar(im1, ax=axes[0], label='V')
axes[0].set_title('Numerical Solution', fontsize=12)
axes[0].set_xlabel('x (m)')
axes[0].set_ylabel('y (m)')
axes[0].set_aspect('equal')

# Analytical solution
im2 = axes[1].pcolormesh(X, Y, phi_analytical, cmap='RdYlBu_r', shading='auto')
fig.colorbar(im2, ax=axes[1], label='V')
axes[1].set_title('Analytical Solution', fontsize=12)
axes[1].set_xlabel('x (m)')
axes[1].set_ylabel('y (m)')
axes[1].set_aspect('equal')

# Error
im3 = axes[2].pcolormesh(X, Y, error, cmap='hot', shading='auto')
fig.colorbar(im3, ax=axes[2], label='|Error| (V)')
axes[2].set_title('Absolute Error', fontsize=12)
axes[2].set_xlabel('x (m)')
axes[2].set_ylabel('y (m)')
axes[2].set_aspect('equal')

plt.tight_layout()
plt.show()

## Summary

In this notebook, we have:

1. **Derived** the Laplace equation from Gauss's law for electrostatics
2. **Implemented** the Jacobi relaxation method to solve the 2D Laplace equation numerically
3. **Computed** the electric field from the potential gradient
4. **Visualized** both the potential distribution and electric field lines
5. **Verified** our numerical solution against the analytical Fourier series solution

### Key Observations:

- The equipotential lines are perpendicular to the electric field lines, as expected from $\mathbf{E} = -\nabla\phi$
- The electric field is strongest near the corners where the boundary conditions change abruptly
- The Jacobi method converges reliably but slowly; more advanced methods like Gauss-Seidel or SOR (Successive Over-Relaxation) would converge faster
- The numerical solution agrees well with the analytical solution, with errors primarily near the boundaries due to the finite Fourier series truncation

### Applications:

The Laplace equation appears in many physical contexts beyond electrostatics:
- Heat conduction (steady-state temperature distribution)
- Fluid dynamics (potential flow)
- Gravitational fields
- Diffusion processes